In [2]:
import bagpy
from bagpy import bagreader
import numpy as np
import pandas as pd

# Read the bag file
bag = bagreader(r"C:\Users\Bryan\Desktop\ece484\Final_Project\rosbag\_2024-10-31-10-09-05.bag")

# Get list of topics
print(bag.topic_table)

# Read point cloud messages
pointcloud_msgs = bag.message_by_topic('/ouster/points')  # adjust topic name as needed
df_pointcloud = pd.read_csv(pointcloud_msgs)

[INFO]  Data folder C:\Users\Bryan\Desktop\ece484\Final_Project\rosbag\_2024-10-31-10-09-05 already exists. Not creating.
                                                Topics  \
0             /camera_fl/arena_camera_node/camera_info   
1    /camera_fl/arena_camera_node/grab_images_raw/s...   
2               /camera_fl/arena_camera_node/image_raw   
3    /camera_fl/arena_camera_node/image_raw/compressed   
4    /camera_fl/arena_camera_node/image_raw/compres...   
..                                                 ...   
242  /zed2/zed_node/stereo_raw/image_raw_color/theo...   
243  /zed2/zed_node/stereo_raw/image_raw_color/theo...   
244                     /zed2/zed_node/temperature/imu   
245                    /zed2/zed_node/temperature/left   
246                   /zed2/zed_node/temperature/right   

                                     Types  Message Count   Frequency  
0                   sensor_msgs/CameraInfo            131    3.180438  
1           actionlib_msgs/GoalStatus

In [53]:
def parse_ouster_frame(row):
    # Get binary data
    binary_data = eval(row['data'])  # Convert string representation to bytes
    
    # Get dimensions
    width = row['width']
    height = row['height']
    point_step = row['point_step']
    
    # Parse to float32 array
    points = np.frombuffer(binary_data, dtype=np.float32)
    
    # Each point has 12 float32 values (48 bytes / 4 bytes per float32 = 12)
    # Typically: x, y, z, intensity, t, reflectivity, ring, noise, range
    points = points.reshape(-1, 12)
    
    # Get just x, y, z, intensity
    xyz_intensity = points
    
    # Reshape to original lidar structure if needed
    xyz_intensity = xyz_intensity.reshape(height, width, 12)
    
    return xyz_intensity

In [54]:
# Process all frames
frames = []
timestamps = []

for idx, row in df_pointcloud.iterrows():
    frame = parse_ouster_frame(row)
    frames.append(frame)
    timestamps.append(row['Time'])

# Convert to numpy array
point_cloud_array = np.array(frames)
timestamps = np.array(timestamps)

In [32]:
point_cloud_array.shape
first_frame = point_cloud_array[0]
x = first_frame[:, :, 0]  # x coordinates
y = first_frame[:, :, 1]  # y coordinates
z = first_frame[:, :, 2]  # z coordinates
intensity = first_frame[:, :, 3]  # intensity values

In [55]:
point_cloud_array.shape

(355, 128, 1024, 12)

In [56]:
flattened_array = point_cloud_array.reshape(355, 131072,12)
np.save('all_points.npy', flattened_array)